In [2]:
import requests
from concurrent.futures import ThreadPoolExecutor
import subprocess
import time
import re
import pandas as pd
import numpy as np
import PyPDF2
import os
import ast
from tqdm import tqdm
from collections import OrderedDict
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import subprocess
import time
import undetected_chromedriver as uc
import re


In [4]:
def get_pdf_links_with_titles(file_path):
    pdf_dict = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line.count(',') < 2:
                continue  # not enough parts, skip
            
            parts = line.split(',')
            scopus_id = parts[0].strip()
            link = parts[-1].strip() if parts[-1].strip() else None
            title = ','.join(parts[1:-1]).strip()  # join all parts between scopus_id and link
            # if link and 'onlinelibrary.wiley.com' in link.lower():
            if link and (('journals.sagepub.com' in link.lower()) or ('onlinelibrary.wiley.com' in link.lower())):
            # if link and 'http' in link.lower():
                pdf_dict[title] = {}
                pdf_dict[title]['link'] = link
                pdf_dict[title]['scopus_id'] = scopus_id
    return pdf_dict

if __name__ == "__main__":
    all_pdfs = {}
    
    file_paths = [
        # './Downloads/retrieved_links_new/retrieved_links_140000-145000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_145000-150000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_150000-155000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_155000-160000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_160000-165000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_165000-170000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_170000-175000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_175000-180000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_180000-185000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_185000-190000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_190000-195000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_195000-200000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_200000-202947.txt'
        './Downloads/retrieved_links_new/retrieved_links_60000-65000.txt',
        './Downloads/retrieved_links_new/retrieved_links_70000-75000.txt',
        './Downloads/retrieved_links_new/retrieved_links_75000-80000.txt',
        './Downloads/retrieved_links_new/retrieved_links_80000-85000.txt',
        './Downloads/retrieved_links_new/retrieved_links_90000-95000.txt',
    ]
    
    for path in file_paths:
        pdf_dict = get_pdf_links_with_titles(path)
        all_pdfs.update(pdf_dict)  # Merges new dict into main one

    print(f"Total unique PDF entries: {len(all_pdfs)}")

Total unique PDF entries: 581


In [6]:
def slice_dict(d, start, end):
    return dict(list(d.items())[start:end])

In [7]:
batch = slice_dict(all_pdfs, 0, 600)
        
all_scopus_ids_in_data = {info['scopus_id'] for info in batch.values()}

remove_df = pd.read_excel('/Users/dtuser/Downloads/Remove_Scopus.xlsx')

# Extract SCOPUS IDs from Excel (assuming the column is named something like 'scopus_id')
remove_scopus_ids = set(remove_df.iloc[:, 1].astype(str).str.extract(r'SCOPUS_ID:(\d+)', expand=False).dropna())


# Check for overlap
overlap_ids = all_scopus_ids_in_data.intersection(remove_scopus_ids)

# Report result
if not overlap_ids:
    print("✅ No SCOPUS IDs in the batch are present in the Excel file.")
else:
    print(f"❌ {len(overlap_ids)} SCOPUS IDs found in both batch and Excel file.")
    print("Conflicting IDs:", overlap_ids)

❌ 92 SCOPUS IDs found in both batch and Excel file.
Conflicting IDs: {'85167718099', '85174643607', '85174644188', '85152260698', '85174650820', '85170688696', '85167599061', '85170432061', '85167676690', '85169907330', '85168356594', '85151343188', '85176498562', '85127561844', '85167563496', '85160269909', '85174490763', '85167605398', '85178584796', '85169895781', '85167609604', '85169886179', '85167601631', '85167577292', '85167600319', '85170062006', '85178631370', '85167614833', '85178607727', '85167674050', '85169891397', '85179606952', '85160943993', '85170409978', '85152473553', '85174657687', '85167625548', '85178577223', '85152001981', '85166019789', '85152206701', '85178587230', '85167606016', '85167626075', '85174654302', '85169894413', '85152127240', '85171148523', '85168671336', '85178597522', '85160926152', '85151485300', '85178618841', '85166369401', '85152298173', '85178566098', '85174650564', '85166369883', '85167570254', '85172071918', '85178552564', '85172011518', 

In [8]:
# Remove entries from batch where info['scopus_id'] is in remove_scopus_ids
filtered_batch = {
    k: v for k, v in batch.items()
    if v.get('scopus_id') not in remove_scopus_ids
}
batch = filtered_batch
all_scopus_ids_in_data = {info['scopus_id'] for info in batch.values()}

remove_df = pd.read_excel('/Users/dtuser/Downloads/Remove_Scopus.xlsx')

# Extract SCOPUS IDs from Excel (assuming the column is named something like 'scopus_id')
remove_scopus_ids = set(remove_df.iloc[:, 1].astype(str).str.extract(r'SCOPUS_ID:(\d+)', expand=False).dropna())


# Check for overlap
overlap_ids = all_scopus_ids_in_data.intersection(remove_scopus_ids)

# Report result
if not overlap_ids:
    print("✅ No SCOPUS IDs in the batch are present in the Excel file.")
else:
    print(f"❌ {len(overlap_ids)} SCOPUS IDs found in both batch and Excel file.")
    print("Conflicting IDs:", overlap_ids)

✅ No SCOPUS IDs in the batch are present in the Excel file.


In [12]:
def open_pdf_in_safari_and_save(pdf_url):
    apple_script = f"""
    tell application "Safari"
    open location "{pdf_url}"
    delay 7 -- Wait for the page to load and for the PDF to render
    end tell
    
    
    tell application "System Events"
        tell process "Safari"
            set frontmost to true  -- Ensure Safari remains the active window
            delay 2.5
        end tell
    
        -- Click the download button inside Safari
        do shell script "cliclick c:1400,128" #---->onlinewiley
        do shell script "cliclick c:1348,129" #---->sage_journal
        delay 1
    end tell
    
    tell application "Safari"
        activate
        tell front window
            close current tab
        end tell
    end tell
    """

    subprocess.run(["osascript", "-e", apple_script], text=True)

In [10]:
def clear_history():
    apple_script = """
    tell application "Safari"
        activate  -- Bring Safari to the front
    end tell

    delay 2  -- Give Safari time to come forward

    tell application "System Events"
        tell process "Safari"
            set frontmost to true  -- Ensure Safari remains the active window
            delay 1
        end tell

        -- Click at position 1
        do shell script "cliclick c:73,1"
        delay 1.5

        -- Click at position 2
        do shell script "cliclick c:115,196"
        delay 1.5

        -- Click at position 3
        do shell script "cliclick c:865,509"
        do shell script "cliclick c:865,509"
        do shell script "cliclick c:846,350"
        do shell script "cliclick c:846,350"
        delay 3
    end tell
    """

    import subprocess
    subprocess.run(["osascript", "-e", apple_script], text=True)


In [ ]:
os.mkdir('/Users/dtuser/Downloads/onlinewiley_pdfs_10Jul')

In [16]:
folder_path = '/Users/dtuser/Downloads/onlinewiley_pdfs_10Jul'

scopus_ids_in_folder = set()
for filename in os.listdir(folder_path):
    if '-' in filename:
        try:
            scopus_id_part = filename.rsplit('-', 1)[-1].split('.')[0]
            scopus_ids_in_folder.add(scopus_id_part)
        except IndexError:
            continue
    else:
        try:
            if int(filename[:-4]):
                scopus_id_part = filename[:-4]
                scopus_ids_in_folder.add(scopus_id_part)
        except:
            continue
        
all_scopus_ids_in_data = {info['scopus_id'] for info in batch.values()}

missing_scopus_ids = all_scopus_ids_in_data - scopus_ids_in_folder
print(len(missing_scopus_ids), len(overlap_ids))
missing_scopus_ids = missing_scopus_ids - overlap_ids

missing_entries = {title: info for title, info in batch.items() if info['scopus_id'] in missing_scopus_ids}
len(scopus_ids_in_folder), len(missing_entries)

42 0


(447, 42)

In [15]:
download_path = '/Users/dtuser/Downloads/onlinewiley_pdfs_10Jul'
count = 0
for title, info in missing_entries.items():
    count += 1
    if count % 40 == 0:
        clear_history()
    link = info['link']
    scopus_id = info['scopus_id']
    try:
        open_pdf_in_safari_and_save(link)
        time.sleep(2)
        files = os.listdir('./Downloads/')
        pdf_files = [f for f in files if f.endswith(".pdf")]
        current_time = time.time()
        recent_files = [
            os.path.join('', f)
            for f in pdf_files
            if current_time - os.path.getctime(os.path.join('./Downloads/', f)) <= 10
        ]
        
        if recent_files == []:
            continue
        safe_title = re.sub(r'</?inf>', '', title)
        safe_title = re.sub(r'[^A-Za-z0-9 ]+', '', safe_title)
        safe_title = safe_title.strip() + '-' + scopus_id
        new_name = os.path.join(download_path, f"{safe_title}.pdf")  # Rename with title
        latest_file = max([os.path.join('./Downloads', f) for f in recent_files], key=os.path.getctime)
        try:
            os.rename(latest_file, new_name)
            print(f"📂 Renamed file to: {new_name}")
        except OSError as e:
            if e.errno == 63:  
                fallback_name = os.path.join(download_path, f"{scopus_id}.pdf")
                os.rename(latest_file, fallback_name)
                print(f"⚠️ Filename too long, saved as: {fallback_name}")
            else:
                raise e 
        continue
    except Exception as e:
        print(f"❌ Error processing ScienceDirect PDF: {e}")
        continue   

📂 Renamed file to: /Users/dtuser/Downloads/onlinewiley_pdfs_10Jul/Analysis of the influence of renewable energy sources on thermal energy generation for isolated power systems Analysis of the influence of renewable energy sources on thermal energy generation for isolated power systems-85074046870.pdf
⚠️ Filename too long, saved as: /Users/dtuser/Downloads/onlinewiley_pdfs_10Jul/85074334763.pdf
⚠️ Filename too long, saved as: /Users/dtuser/Downloads/onlinewiley_pdfs_10Jul/85087464239.pdf


KeyboardInterrupt: 

In [ ]:
count